## Figure of merit

**The notebook presents a function which take as input the bundleDicts from the metric and it builds a bar plot with the rank of all the opsims. The rank is estimated summing the metric value all over the footprint.**

In [ ]:
%matplotlib inline
import sys,os, glob
import numpy as np
import math
from opsimUtils import *
import matplotlib.pyplot as plt
import pandas as pd
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.metricBundles as metricBundles
import lsst.sims.maf.db as db
import lsst.sims.maf.plots as plots
from lsst.sims.maf.metrics import BaseMetric
from lsst.sims.maf.utils import m52snr, sigma_slope, astrom_precision
from builtins import zip

- your_username has to be set to be able to access to the proper directory.

- outDir has to be set to be able to access to the MAFOutput file.

In [ ]:
your_username = ''
if your_username == '': # do NOT put your username here, put it in the cell at the top of the notebook.
    raise Exception('Please provide your username!  See the top of the notebook.')
    
outDir = '/home/idies/workspace/Storage/{}/persistent/MAFOutput/MetricData'.format(your_username)

resultDbs = getResultsDbs(outDir)

# the following line will be useful if you did not run MAF on all 75 opsims
runNames = list(resultDbs.keys())
bundleDicts = {}
for runName in runNames:
       bundleDicts[runName] = bundleDictFromDisk(resultDbs[runName], runName, outDir)


nomalization factors and/or tresholds for the FOM and MAF values can be set in the function to plot the selection we want to show

In [2]:
def FOM(bundleDicts, norm=None, treshold_min =None, treshold_max =None, **karg):
    data_bundle = {}
    data = {}
    FOMs = []
    runNames= list(bundleDicts.keys())
    metrics = bundleDicts[runNames[0]].keys()
    for metric in metrics:
        data_bundle[metric]={}
        data[metric]={}
        FOM=[]
        for runName in runNames: 
            data_bundle[metric][runName]= pd.DataFrame(bundleDicts[runName][metric].metricValues[np.where(bundleDicts[runName][metric].metricValues.mask == False)])[0]
            data[metric][runName]= [] 
            for i in range(len(data_bundle[metric][runName])):        
                if data_bundle[metric][runName][i]:
                    data[metric][runName].append(data_bundle[metric][runName][i])
            data[metric][runName]=np.array(data[metric][runName][np.where(np.isfinite(data[metric][runName]))]) 
            if treshold_min !=None and treshold_max !=None :
                ID_fom=np.where((data[metric][runName]>treshold_min) & (data[metric][runName]<treshold_max))
            else:
                if treshold_min !=None:
                    ID_fom=np.where(data[metric][runName]>treshold_min)
                elif treshold_max !=None :
                    ID_fom=np.where(data[metric][runName]<treshold_max)
                else:
                    ID_fom=np.repeat(True, len(data[metric][runName]))
            if norm =='std':
                N= np.std(data[metric][runName][ID_fom])
            else:
                N=np.repeat(1,len(data[metric][runName][ID_fom]))
            fom = np.array(data[metric][runName][ID_fom])*N
            FOM.append(fom.sum())
        FOM=np.array(FOM)
        FOMs.append(FOM) 
    
    name_tick=[]
    for key in bundleDicts.keys():
        n= key.split('_')
        if 'v1.4' in n:
            n.remove('v1.4')
        else:
            word=[]
            for s in n[-2]:
                word.append(s)
            word.remove('v')
            word.remove('1')
            word.remove('.')
            word.remove('4')
            sub =''
            n[-2]= sub.join(word)
        n.remove('10yrs')
        s=' '
        name_tick.append(s.join(n))
    
    import matplotlib.cm as cm
    plt.rcdefaults()
    fig, ax = plt.subplots(figsize=(10,30))
    y_pos = np.arange(len(bundleDicts.keys()))
    colorlist = cm.rainbow(np.linspace(0, 1, len(FOMs)))
    d= np.zeros(len(runNames))
    dplt=[]
    FOM_labels= list(bundleDicts[runNames[0]].keys())
    for i, FOM in enumerate(FOMs):
        d =+ FOM
        dplt.append(d)
    print(dplt)
    idx=np.argsort(dplt[-1])
    idx=idx[::-1]
    for i in reversed(range(len(FOMs))):
        ax.barh(y_pos,dplt[i], align='center',color=colorlist[i],label=FOM_labels[i], **karg)
    ax.set_yticks(y_pos)
    ax.set_yticklabels(name_tick, size=20)
    ax.invert_yaxis()  # labels read top-to-bottom
    ax.set_xlabel('FoM', size=20)
    ax.legend(bbox_to_anchor=(1., 1.),fontsize=15)
    if filename!=None:
        plt.savefig(filename+'.png')
    plt.close()
    return fig